In [3]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('popular_comment_Bolt_annotate_Merged.csv')

# Create a new column based on irony 1 and irony 2 columns
df['true_label'] = df[['irony 1', 'irony 2']].max(axis=1)

# Remove rows where either remove 1 or remove 2 contains a '1' or '2'
df = df[(df['remove 1'] != 1) & (df['remove 1'] != 2) & (df['remove 2'] != 1) & (df['remove 2'] != 2)]

# Write the modified DataFrame to a new CSV file
df.to_csv('true_label.csv', index=False)

In [4]:
import numpy as np
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-sarcasm-twitter")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-sarcasm-twitter")

def eval_conversation(text):
  input_ids = tokenizer.encode(text + '</s>', return_tensors='pt',  truncation=True)
  output = model.generate(input_ids=input_ids, max_length=3)
  dec = [tokenizer.decode(ids) for ids in output]
  label = dec[0]
  return label[6:]

# Load CSV data
csv_file_path = 'true_label.csv'
data = pd.read_csv(csv_file_path)

# Assuming 'text' column contains the text data and 'truth_label' column contains the true labels 
X_test = data['text']
y_test = data['true_label']

# Initialize an empty list to store predictions
predictions = []

print('Running data through pre-trained model...')

# Iterate through each row (line) in the text column of the DataFrame
for text in X_test:
    predicted_label = eval_conversation(text)
    # print(predicted_label)
    if predicted_label == 'derison':
        predictions.append(1)
    else:
        predictions.append(0)
    
# Add the list of predictions as a new column in the DataFrame
data['predicted_label'] = predictions

# Save the updated DataFrame to a new CSV file
output_csv_file_path = 'predication.csv'
data.to_csv(output_csv_file_path, index=False)

print(f"Predictions have been added as a new column and saved to {output_csv_file_path}")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
C:\Users\Admin\anaconda3\Lib\site-packages\transformers\models\auto\modeling_auto.py:1682: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Running data through pre-trained model...
Predictions have been added as a new column and saved to predication.csv


In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, f1_score

# Load the CSV file that contains the true labels and predicted labels
csv_file_path = 'predication.csv'
data = pd.read_csv(csv_file_path)

# Assuming 'label' column contains the true labels and 'predicted_label' column contains the model's predictions
y_true = data['true_label']
y_pred = data['predicted_label']

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision and recall for the sarcastic class
# Define the positive label as the sarcastic label (usually 'sarcastic' or 1, adjust as needed)
positive_label = 1  # Adjust this as per your data

# Calculate precision
precision = precision_score(y_true, y_pred, pos_label=positive_label)
print("Precision:", precision)

# Calculate recall
recall = recall_score(y_true, y_pred, pos_label=positive_label)
print("Recall:", recall)

# Calculate F1-score
f1 = f1_score(y_true, y_pred, pos_label=positive_label)
print("F1-score:", f1)

#Print classification report for more detailed evaluation
print("\nClassification Report:")
print(classification_report(y_test, y_true))

Accuracy: 0.5219611848825332
Precision: 0.11042944785276074
Recall: 0.6206896551724138
F1-score: 0.1875

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       892
           1       1.00      1.00      1.00        87

    accuracy                           1.00       979
   macro avg       1.00      1.00      1.00       979
weighted avg       1.00      1.00      1.00       979

